### Simple chain.

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = HuggingFacePipeline.from_model_id(
    model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task = "text-generation",
    device = -1,
    pipeline_kwargs={"temperature":0.7}
)

model = ChatHuggingFace(llm=llm)

template = PromptTemplate(
    template="Write a 5 interesting fact about {topic}. \n"
    "Write all the facts line by line and number them accordingly. \n"
    "Example: \n"
    "1. Proteins are important lifeline for life. \n"
    "2. There are 20 types of proteins. \n"
    "3. .............",
    input_variables=["topic"]
)

parser = StrOutputParser()

chain = template | model | parser

result = chain.invoke({"topic": "Taj Mahal"})

print(result)

/Users/arjun/miniforge3/envs/ai/lib/python3.11/site-packages/torchvision/io/image.py:14: UserWarning: Failed to load image Python extension: 'dlopen(/Users/arjun/miniforge3/envs/ai/lib/python3.11/site-packages/torchvision/image.so, 0x0006): Library not loaded: @rpath/libjpeg.9.dylib
  Referenced from: <EB3FF92A-5EB1-3EE8-AF8B-5923C1265422> /Users/arjun/miniforge3/envs/ai/lib/python3.11/site-packages/torchvision/image.so
  Reason: tried: '/Users/arjun/miniforge3/envs/ai/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/arjun/miniforge3/envs/ai/lib/python3.11/site-packages/torchvision/../../../libjpeg.9.dylib' (no such file), '/Users/arjun/miniforge3/envs/ai/lib/python3.11/lib-dynload/../../libjpeg.9.dylib' (no such file), '/Users/arjun/miniforge3/envs/ai/bin/../lib/libjpeg.9.dylib' (no such file)'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your e

<|user|>
Write a 5 interesting fact about Taj Mahal. 
Write all the facts line by line and number them accordingly. 
Example: 
1. Proteins are important lifeline for life. 
2. There are 20 types of proteins. 
3. .............</s>
<|assistant|>
Here's a 5-fact about Taj Mahal:

1. Taj Mahal is made of white marble.
2. The Taj Mahal is made of 24 karat gold.
3. The Taj Mahal is the most expensive building project to be completed in history.
4. The Taj Mahal is the only structure in the world with four courtyards.
5. The Taj Mahal is the most beautiful and grandest monument in the world.

Facts numbered 1 through 4 are listed in order, while numbers 5 and up are listed in reverse order.


### Sequential Chain. 

In [19]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = HuggingFacePipeline.from_model_id(
    model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task = "text-generation",
    device = -1,
    pipeline_kwargs={"temperature":0.7, "max_new_tokens": 2024}
)

model = ChatHuggingFace(llm=llm)

prompt1 = PromptTemplate(
    template="Generate very short 5 line report on {topic}",
    input_variables=["topic"]
)

prompt2 = PromptTemplate(
    template="Write a 5 interesting fact about {topic}. \n"
    "Write all the facts line by line and number them accordingly. \n"
    "Example: \n"
    "1. Proteins are important lifeline for life. \n"
    "2. There are 20 types of proteins. \n"
    "3. .............",
    input_variables=["text"]
)

parser = StrOutputParser()

chains = prompt1 | model | parser | prompt2 | model | parser

result = chains.invoke({"topic" : "Cricket worldwide"})

print(result)

Device set to use cpu


<|user|>
Write a 5 interesting fact about <|user|>
Generate very short 5 line report on Cricket worldwide</s>
<|assistant|>
Introduction:

The Cricket worldwide is a professional cricket league established in 1971, which is currently run by the International Cricket Council (ICC). The league comprises of a number of countries that are divided into regions like the ACC, CPL, ODI, T20, and T20I. The league has grown significantly in recent years, with the addition of a number of new teams and leagues. This report reviews the history, evolution, and current state of the Cricket worldwide.

History:

The Cricket worldwide was established in 1971 as the predecessor of the current ICC Cricket World Cup. The league was initially played between seven countries, including Australia, England, India, and Pakistan. The inaugural tournament was held in India and Sri Lanka, and the winner was the host country, India. The tournament was a one-off event, and it was not held again until 1976.

In 1977,

### Parallel chains. 

In [28]:
from langchain_openai import ChatOpenAI
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from dotenv import load_dotenv
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

load_dotenv()

model_gpt = ChatOpenAI()

llm = HuggingFacePipeline.from_model_id(
    model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task = "text-generation",
    device = -1,
    pipeline_kwargs={
        "temperature":0.7, 
        "max_new_tokens": 512,
        "return_full_text": False}
)

model_free = ChatHuggingFace(llm=llm)

template_1 = PromptTemplate(
    template = "Give me 5-line short summary from this:\n{text}",
    input_variables = ["text"]
)

template_2 = PromptTemplate(
    template = "Make 3 questions and answers from this:\n{text}",
    input_variables = ["text"]
)

template_3 = PromptTemplate(
    template = "Merge these notes and Q&A into a single study note:\n\nNotes:\n{notes}\n\nQ&A:\n{quesans}",
    input_variables = ["notes","quesans"]
)

chain_parallel = RunnableParallel({
    "notes": template_1 | model_free | parser,
    "quesans": template_2 | model_gpt | parser
})

parser = StrOutputParser()

chain_merge = template_3 | model_gpt | parser

chain = chain_parallel | chain_merge

text = """ 
Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression and outliers detection.
The advantages of support vector machines are:
Effective in high dimensional spaces.
Still effective in cases where number of dimensions is greater than the number of samples.
Uses a subset of training points in the decision function (called support vectors), so it is also memory efficient.
Versatile: different Kernel functions can be specified for the decision function. Common kernels are provided, but it is also possible to specify custom kernels.
The disadvantages of support vector machines include:
If the number of features is much greater than the number of samples, avoid over-fitting in choosing Kernel functions and regularization term is crucial.
SVMs do not directly provide probability estimates, these are calculated using an expensive five-fold cross-validation (see Scores and probabilities, below).
The support vector machines in scikit-learn support both dense (numpy.ndarray and convertible to that by numpy.asarray) and sparse (any scipy.sparse) sample vectors as input. However, to use an SVM to make predictions for sparse data, it must have been fit on such data. For optimal performance, use C-ordered numpy.ndarray (dense) or scipy.sparse.csr_matrix (sparse) with dtype=float64.
"""

result = chain.invoke({"text" : text})

print(result)

Device set to use cpu


Support vector machines (SVMs) are a set of supervised learning methods used for classification, regression, and outlier detection. They have advantages such as being effective in high-dimensional spaces, memory-efficient by using a subset of training points in the decision function, and the ability to use support vectors as decision functions. However, there are some disadvantages to SVMs. They require a stable and efficient kernel function for effective decision making, need good performance on five-fold cross-validation for calculating scores and probabilities, and perform best when trained on sparse data rather than dense sample vectors.

Q&A:
1. What are some advantages of support vector machines (SVMs)?
- Effective in high dimensional spaces
- Memory efficient
- Versatile: can specify different Kernel functions for decision making
- Can handle cases where number of dimensions is greater than number of samples

2. What are some disadvantages of support vector machines (SVMs)?
- Ri

### Conditional Chain.

In [ ]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from pydantic import BaseModel, Field
from langchain_core.runnables import RunnableBranch, RunnableParallel, RunnableLambda
from typing import Literal
from langchain_openai import ChatOpenAI
"""
llm = HuggingFacePipeline.from_model_id(
    model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0",
    task = "text-generation",
    device = -1,
    pipeline_kwargs={
        "temperature":0.7, 
        "max_new_tokens": 512,
        "return_full_text": False}
)
"""

model = ChatOpenAI()

class Sentiment_vali(BaseModel):

    sentiment: Literal["positive", "negative"] = Field(description = "Predict the sentiment of the review.")

sentiment_parser = PydanticOutputParser(pydantic_object=Sentiment_vali)

parser = StrOutputParser()

template = PromptTemplate(
    template = "Give me the sentiment of this review in positive and negative format. \n {review}. {format_instructions}",
    input_variables = ["review"],
    partial_variables = {'format_instructions': sentiment_parser.get_format_instructions()}
)

chain_sentiment = template | model | sentiment_parser

template_2 = PromptTemplate(
    template = "Write an appropriate response to this positive sentiment. {sentiment}",
    input_variables = ['sentiment']
)

'negative'

In [16]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser, PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Literal
from langchain_core.runnables import RunnableBranch, RunnableParallel, RunnableLambda
from dotenv import load_dotenv

load_dotenv()
model = ChatOpenAI()

parser = StrOutputParser()

class sentiment_one_word(BaseModel):

    sentiment: Literal["positive", "negative"] = Field(description="Predict the sentiment of the review in one word.")

parser_two = PydanticOutputParser(pydantic_object= sentiment_one_word)

template_one = PromptTemplate(
    template = "Give me the sentiment of this review in one word positive or negative. \n {review}. {format_instructions}",
    input_variables=["review"],
    partial_variables={'format_instructions': parser_two.get_format_instructions()}
)

chain_sentiment_one = template_one | model | parser_two

#result = chain_sentiment_one.invoke({"review": "The product quality is excellent and I am very satisfied with my purchase."})

template_positive = PromptTemplate(
    template = "Write an appropriate response to this positive sentiment. {sentiment}",
    input_variables = ['sentiment']
)

template_negative = PromptTemplate(
    template = "Write an appropriate response to this negative sentiment. {sentiment}",
    input_variables = ['sentiment']
)

chain_positive = template_positive | model | parser
chain_negative = template_negative | model | parser

chain = RunnableBranch(
    (lambda x: x.sentiment == 'positive', chain_positive),
    (lambda x: x.sentiment == 'negative', chain_negative),
    RunnableLambda(lambda x: "Sentiment not recognized.")
)

full_chain = chain_sentiment_one | chain
result = full_chain.invoke({"review": "The quality of this phone is terrible. I didn't like the performance and battery life at all."})
print(result)

I'm sorry to hear that you are feeling negative about the situation. Is there anything specific that is bothering you that you would like to talk about? Let me know how I can help or support you.
